# Import Library

In [1]:
import re
import string
import numpy as np
import pandas as pd

import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk
from nltk.tokenize import word_tokenize

import pickle

# Pre-processing

In [2]:
def get_translate_text(text):
    keyword = ['(Diterjemahkan oleh Google)', '(Asli)']
    if keyword[0] in text and keyword[1] in text:
        text = text[text.find(keyword[0]) + len(keyword[0]):text.rfind(keyword[1])]
    elif keyword[0] in text and keyword[1] not in text:
        text = text[text.find(keyword[0]) + len(keyword[0]):]
    return text

def reduction_spelling_error(text):
    if re.search(r'([a-zA-Z])\1{2,}', text): # Spell
        text = re.sub(r'([a-zA-Z])\1{2,}','\\1', text)
    return text

def ws_remove(text):
    text = re.sub(r'\s+', ' ', text)
    return text

def pt_remove(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'[0-9]', ' ', text)

    return text

def stop(text):
    stop_factory = StopWordRemoverFactory().get_stop_words() #load default stopword
    more_stopword = ['daring', 'online','yg'] #menambahkan stopword

    data = stop_factory + more_stopword #menggabungkan stopword

    dictionary = ArrayDictionary(data)
    str = StopWordRemover(dictionary)
    return str.remove(text)

def stemmer(text):
    factory = StemmerFactory()
    stemmerr = factory.create_stemmer()
    return stemmerr.stem(text)

In [3]:
# convert emoji
# import emoji
def convert_unicode(text):
    text= repr(text.encode('ascii',errors='backslashreplace'))
    text= text.replace("\\U000"," 0x")
    # text = re.sub('\\', '', text)
    text=text[2:-1]
    text_tokens = text.split(" ")
    new_text = ""
    for i in text_tokens:
        # print(i[:1])
        if i[:1] == '\\':
            continue
        else:
            new_text += " " + i
    return new_text
# df['text_unicode'] = df['text_ws_remove'].apply(convert_unicode)
df_emoji= pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTDw-1PPBpNDwXch3yb3ApjszyJ1dX5iAOKQWnjDsdEqpzVAaiM9cOduOk63A8WGSTwz_HFZbI5ZUGR/pub?gid=1445261330&single=true&output=csv')# df_emoji.head()

# create a list of our conditions
conditions = [
    (df_emoji['Negative'] > df_emoji['Neutral']) & (df_emoji['Negative'] >= df_emoji['Positive']),
    (df_emoji['Negative'] < df_emoji['Neutral']) & (df_emoji['Neutral'] >= df_emoji['Positive']),
    (df_emoji['Neutral'] < df_emoji['Positive']) & (df_emoji['Negative'] <= df_emoji['Positive'])
    ]

# create a list of the values we want to assign for each condition
values = ['negatif', 'netral', 'positif']

# create a new column and use np.select to assign values to it using our lists as arguments
df_emoji['sentimen'] = np.select(conditions, values)

# display updated DataFrame
# df_emoji.head()
#drop unused column
df_emoji=df_emoji.drop(columns=['Emoji',	'Occurrences',	'Position'	,'Negative',	'Neutral',	'Positive',	'Unicode name',	'Unicode block'])
# df_emoji
# df_emoji['Unicode codepoint'] = df_emoji['Unicode codepoint'].apply(clean_text)
df_emoji=df_emoji.set_index(['Unicode codepoint'])
# df_emoji.head()
dict_emoji=df_emoji.to_dict()
dict_emoji=dict_emoji['sentimen']
# dict_emoji
# text = 'Perjalanan yg bnr" bikin Syahduu ,, dr Tempat parkir ke lokasi Curug ,, Lelahh pun terasa hilang ketika sampe lokasi ,, 🥰🥰 …'
def convert_emoji(text):
    text_tokens = text.split(" ")
    new_text = ""
    for i in text_tokens:
        if i[:2] =='0x':
            if i in dict_emoji:
                new_text +=  " " +dict_emoji[i]
            else:
              i=''
              new_text+=i
        else:
            new_text += " " + i
    return new_text

# New Text

In [4]:
teks='''Barusan dari situ dan pas ke toiletnya ampun kotor banget,dan bau pesing,tidak terawat padahal berbayar.
Kalah sama toilet pom bensin yang gratis.
Udah gitu pemandangannya biasa aja.
'''

In [5]:
teks='Barusan dari situ dan pas ke toiletnya ampun kotor banget,dan bau pesing,tidak terawat padahal berbayar'

# Pre-processing New Text

In [6]:
teks=get_translate_text(teks)
teks=reduction_spelling_error(teks)
teks=convert_unicode(teks)
teks=convert_emoji(teks)
teks=pt_remove(teks)
teks=teks.lower()
teks=stop(teks)
teks=stemmer(teks)
teks

'barusan situ pas toilet ampun kotor banget bau pesing awat bayar'

In [7]:
len(teks)

64

In [8]:
print([teks])

['barusan situ pas toilet ampun kotor banget bau pesing awat bayar']


# TF-IDF

In [11]:
with open("../model/vectorizer.pkl", 'rb') as f:
    vectorizer = pickle.load(f)

C:\Users\Fakhri\AppData\Local\Temp\ipykernel_7332\262096972.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  vectorizer = pickle.load(f)


In [12]:
# teks=df_test.values.tolist()
# teks=['kali','dua','seperti','biasa','sejuk','ramai,jalan,puncak,sedikit,kurang,bagus,tiket,weekend,40an,rb,1,orang,1,sepeda,motor,fasilitas,perlu,tingkat,untuk,beberapa,toilet,itu,udh,rapi,aman']
teks=vectorizer.transform([teks])
# teks


In [13]:
teks

<1x390 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

# Convert to DataFrame

In [14]:
df_teks = pd.DataFrame(teks.toarray(), columns=vectorizer.get_feature_names_out())
print(df_teks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 390 entries, acara to yg
dtypes: float64(390)
memory usage: 3.2 KB
None


# Import Model

In [15]:
with open("../model/model.pkl", 'rb') as f:
    clf = pickle.load(f)

# Predict

In [16]:
sample_predict = clf.predict(df_teks)
sample_predict

array([[0, 2, 0]], dtype=int64)

In [17]:
sample_proba = clf.predict_proba(df_teks)
sample_proba

[array([[0.9971214 , 0.00124123, 0.00163737]], dtype=float32),
 array([[0.01576329, 0.01301333, 0.9712234 ]], dtype=float32),
 array([[0.5490253 , 0.09777901, 0.35319573]], dtype=float32)]

In [20]:
len(sample_proba)

3

In [18]:
result = [
    {
        'label': 'aksesibilitas',
        'score': {
            'neutral': round(sample_proba[0][0][0], 3),
            'positive': round(sample_proba[0][0][1], 3),
            'negative': round(sample_proba[0][0][2], 3)
        }
    },
    {
        'label': 'fasilitas',
        'score': {
            'neutral': round(sample_proba[1][0][0], 3),
            'positive': round(sample_proba[1][0][1], 3),
            'negative': round(sample_proba[1][0][2], 3)
        }
    },
    {
        'label': 'aktivitas',
        'score': {
            'neutral': round(sample_proba[2][0][0], 3),
            'positive': round(sample_proba[2][0][1], 3),
            'negative': round(sample_proba[2][0][2], 3)
        }
    },
]

In [19]:
result

[{'label': 'aksesibilitas',
  'score': {'neutral': 0.997, 'positive': 0.001, 'negative': 0.002}},
 {'label': 'fasilitas',
  'score': {'neutral': 0.016, 'positive': 0.013, 'negative': 0.971}},
 {'label': 'aktivitas',
  'score': {'neutral': 0.549, 'positive': 0.098, 'negative': 0.353}}]

# Convert Predict

In [15]:
sampel=sample_predict.tolist()

In [16]:
print(sampel)

[[0, 2, 2]]


In [ ]:
converted=[]
for i in sampel:
    for j in i:
        if j == 0:
            converted.append('Netral')
        elif j == 1: 
            converted.append('Positif')
        else:
            converted.append('Negatif')
print("Aksesibilitas\t",converted[0])
print("Fasilitas\t",converted[1])
print("Aktivitas\t",converted[2])